In [1]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
import plotly.express as px
import branca.colormap as cm
from IPython.display import HTML
from folium.plugins import MousePosition
import warnings
from folium import Popup
warnings.filterwarnings('ignore')

In [2]:
data_collar = pd.read_csv("datasets/Collar.csv")
data_litologi = pd.read_csv("datasets/Litologi.csv")
data_assay = pd.read_csv("datasets/Assay.csv")

In [3]:
data_collar['Hole_id'] = data_collar['Hole_id'].str.extract('(\d+)', expand=False).astype(int)
data_assay['Hole_id'] = data_assay['Hole_id'].str.extract('(\d+)', expand=False).astype(int)
data_litologi['hole_id'] = data_litologi['hole_id'].str.extract('(\d+)', expand=False).astype(int)

In [4]:
new_data = pd.concat([
    data_assay[['Hole_id']],
    data_litologi[['dept_from', 'Dept_to', 'Litho']].rename(columns={
        'dept_from': 'Depth_from',
        'Dept_to': 'Depth_to'
    }),
    data_assay[[' Ni']].rename(columns={' Ni': 'Ni'})
], axis=1)

# save csv
new_data.to_csv('datasets/new_data.csv', index=False)

# display the new data
new_data.head()

,Hole_id,Depth_from,Depth_to,Litho,Ni
0,1.0,0.0,1.0,LIM,0.50
1,1.0,1.0,2.0,LIM,0.72
2,1.0,2.0,3.0,LIM,0.89
3,1.0,3.0,4.0,LIM,0.98
4,1.0,4.0,5.0,LIM,0.90


In [5]:
# Filter SAP lithology data
sap_data = new_data[new_data['Litho'] == 'SAP'].copy()

# Group by Hole_id to get depth and Ni details
aggregated_data = sap_data.groupby('Hole_id').agg(
    Depth_from = ('Depth_from', 'first'),
    Depth_to = ('Depth_to', 'last'),
    Avg_Ni = ('Ni', 'mean'),
    Min_Ni = ('Ni', 'min'),
    Max_Ni = ('Ni', 'max')
).reset_index()

# Calculate total SAP thickness
aggregated_data['Total_SAP'] = aggregated_data['Depth_to'] - aggregated_data['Depth_from']

# save to .csv file
aggregated_data.to_csv('datasets/aggregated_data.csv', index=False)

aggregated_data.head()

,Hole_id,Depth_from,Depth_to,Avg_Ni,Min_Ni,Max_Ni,Total_SAP
0,1.0,11.0,16.0,1.601667,1.03,2.08,5.0
1,2.0,9.0,24.0,2.294000,1.38,3.07,15.0
2,3.0,4.4,9.0,1.964000,0.83,2.53,4.6
3,4.0,8.0,13.0,1.614000,1.25,2.17,5.0
4,5.0,5.0,10.0,1.724000,1.50,2.13,5.0


In [15]:
# Combine all relevant columns for plotting data
plot_data = pd.concat([
    aggregated_data[['Hole_id']],
    data_collar[['X', 'Y']],
    aggregated_data[['Depth_from', 'Depth_to', 'Total_SAP', 'Avg_Ni', 'Min_Ni', 'Max_Ni']]
], axis= 1)

# save to .csv file
plot_data.to_csv('datasets/plot_data.csv', index=False)

plot_data

,Hole_id,X,Y,Depth_from,Depth_to,Total_SAP,Avg_Ni,Min_Ni,Max_Ni
0,1.0,-2.507508,121.513665,11.0,16.00,5.00,1.601667,1.03,2.08
1,2.0,-2.507511,121.512784,9.0,24.00,15.00,2.294000,1.38,3.07
2,3.0,-2.507511,121.511902,4.4,9.00,4.60,1.964000,0.83,2.53
3,4.0,-2.507515,121.510981,8.0,13.00,5.00,1.614000,1.25,2.17
4,5.0,-2.507488,121.510041,5.0,10.00,5.00,1.724000,1.50,2.13
...,...,...,...,...,...,...,...,...,...
120,121.0,-2.510636,121.507853,10.0,17.00,7.00,1.635000,1.18,2.12
121,122.0,-2.510652,121.508255,9.0,11.33,2.33,1.512000,0.66,2.10
122,123.0,-2.510583,121.508693,8.0,14.00,6.00,2.520000,1.32,3.92
123,124.0,-2.510650,121.509187,8.0,18.00,10.00,2.006154,0.60,2.93


In [7]:
# plt.figure(figsize=(15, 8))
# plt.scatter(data_collar['Y'], data_collar['X'], c='blue', alpha=0.5)
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')
# plt.title('Peta titik dari data collar')
# plt.grid(True)
# plt.show()

In [66]:
# Define bounds for map view
min_lat, max_lat = plot_data['X'].min(), plot_data['X'].max()
min_lon, max_lon = plot_data['Y'].min(), plot_data['Y'].max()

# Define value ranges for color mapping
Min_Avg_Ni = plot_data['Avg_Ni'].min()
Max_Avg_Ni = plot_data['Avg_Ni'].max()

# Create base map
map = folium.Map(
    zoom_start=17,
    min_zoom=10,
    control_scale= True,
    font_size='1.5rem'
    )
# Fitting map to data bounds
map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])

# Create color scale
colormap = cm.linear.YlOrRd_09.scale(Min_Avg_Ni, Max_Avg_Ni)
colormap.caption = ' Avg. Nickel Value Scale'
colormap.add_to(map)

# Add mouse position to bottom right of map
mouse_position = MousePosition(
    position='bottomright',
    separator=' : ',
    prefix='Lat | Lon:',
    lat_formatter="function(num) {return L.Util.formatNum(num, 10);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 10);}",
)
mouse_position.add_to(map)

# Add circle markers for each row
for _, row in plot_data.iterrows():
    # color for the circle
    color = colormap(row['Avg_Ni'])
    # radius size of each circle
    radius = 2 + row['Total_SAP'] * 0.7

    folium.CircleMarker(
        location=[row['X'], row['Y']],  
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=1,
        tooltip=f"<b>BH:</b> {row['Hole_id']:.0f}<br>",
        popup=Popup(
        html=(
            f"<b>BH:</b> {row['Hole_id']:.0f}<br>"
            f"<b>Lat:</b> {row['X']:.5f}<br>"
            f"<b>Lon:</b> {row['Y']:.5f}<br>"
            f"<b>SAP:</b> {row['Total_SAP']:.2f} meter<br>"
            f"<b>Avg. Ni:</b> {row['Avg_Ni']:.2f}<br>"
            f"<b>Min. Ni:</b> {row['Min_Ni']:.2f}<br>"
            f"<b>Max. Ni:</b> {row['Max_Ni']:.2f}<br>"
        ),
        max_width=200  
    )
    ).add_to(map)

# Displaying the map
# The map will fill the entire output cell, so it's customized through css
display(HTML('<div style="width: 1000px; height: 700px;">'+map._repr_html_()+'</div>'))
# map # uncomment this to use show full size of the map

In [9]:
# map = folium.Map(zoom_starts = 15)

# locations = []
# for _, row in data_collar.iterrows():
#     location = [row['X'], row['Y']]
#     locations.append(location)
#     folium.CircleMarker(location=location, radius=3, color='blue', fill=True, fill_opacity=0.7).add_to(map)

# # Fit bounds ke semua lokasi
# map.fit_bounds(locations)
# map.save("collar_map.html")

In [10]:
# fig = px.scatter_mapbox(
#     data_collar,
#     lat="X",
#     lon="Y",
#     zoom=16,
#     height=600
# )

# fig.update_layout(mapbox_style="open-street-map")

# fig.show()